In [1]:
import pandas as pd
from pandas_datareader import data as web

In [8]:
fred_series = {
    # Growth & Income
    "Real GDP growth (q/q SAAR)": "A191RL1Q225SBEA",
    "Nominal GDP growth (q/q SAAR)": "A191RP1Q027SBEA",
    "Real disposable income (level)": "DSPIC96",  # we'll compute % change
    "Nominal disposable income (level)": "DSPI",  # we'll compute % change

    # Labor Market & Prices
    "Unemployment rate": "UNRATE",  # monthly -> quarterly avg
    "CPI (price level)": "CPIAUCSL",  # monthly -> quarterly pct-change

    # Interest Rates
    "3-month Treasury rate": "TB3MS",  # monthly
    "5-year Treasury yield": "GS5",    # monthly
    "10-year Treasury yield": "GS10",  # monthly
    "BBB corporate yield": "BAA",      # monthly
    "Mortgage rate (30yr fixed)": "MORTGAGE30US",  # weekly
    "Prime rate": "DPRIME",            # daily

    # Asset Prices
    "Dow Jones Total Stock Market Index": "BOGZ1FL073164013Q",  # quarterly
    "House Price Index": "USSTHPI",  # quarterly
    "Commercial Real Estate Price Index": "COMREPUSQ159N",  # quarterly
    "VIX (market volatility index)": "VIXCLS"  # daily
}

start_date = "1990-01-01"
raw_data = {}

# Pull all data series
for label, fred_id in fred_series.items():
    try:
        df = web.DataReader(fred_id, data_source='fred', start=start_date)
        raw_data[label] = df
    except Exception as e:
        print(f"Error loading {label}: {e}")

# Combine all series into a single DataFrame, aligned on dates
all_data = pd.concat(raw_data.values(), axis=1)
all_data.columns = raw_data.keys()

# Resample to quarterly
quarterly_data = pd.DataFrame(index=pd.date_range(start='1990-03-31', end=all_data.index.max(), freq='QE-DEC'))

for col in all_data.columns:
    series = all_data[col]
    freq = series.index.inferred_freq
    if freq == 'Q':
        quarterly_data[col] = series  # already quarterly
    else:
        quarterly_data[col] = series.resample('QE-DEC').mean()

# Compute % change for income and CPI levels
quarterly_data["Real disposable income growth"] = quarterly_data["Real disposable income (level)"].pct_change() * 100
quarterly_data["Nominal disposable income growth"] = quarterly_data["Nominal disposable income (level)"].pct_change() * 100
quarterly_data["CPI inflation rate"] = quarterly_data["CPI (price level)"].pct_change() * 100

# Drop original level columns
quarterly_data.drop(columns=[
    "Real disposable income (level)",
    "Nominal disposable income (level)",
    "CPI (price level)"
], inplace=True)

# Final cleanup: sort and drop any rows with all NaNs
quarterly_data = quarterly_data.sort_index().dropna(how='all')

# Preview
quarterly_data.tail()

# Optional: Save to CSV
#quarterly_data.to_csv("fred_quarterly_data.csv")

,Real GDP growth (q/q SAAR),Nominal GDP growth (q/q SAAR),Unemployment rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,BBB corporate yield,Mortgage rate (30yr fixed),Prime rate,Dow Jones Total Stock Market Index,House Price Index,Commercial Real Estate Price Index,VIX (market volatility index),Real disposable income growth,Nominal disposable income growth,CPI inflation rate
2024-03-31,1.6,4.7,3.833333,5.233333,4.123333,4.160000,5.733333,6.748462,8.500000,52402860.0,662.70,-10.000029,13.712540,1.366710,2.223763,0.913816
2024-06-30,3.0,5.6,4.000000,5.243333,4.460000,4.443333,5.923333,6.995385,8.500000,53915680.0,679.36,-13.388868,13.980308,0.259954,0.888897,0.682157
2024-09-30,3.1,5.0,4.166667,4.990000,3.790000,3.946667,5.620000,6.508462,8.437500,57046410.0,686.41,-10.472797,17.066667,0.052580,0.435510,0.347178
2024-12-31,2.4,4.8,4.133333,4.400000,4.130000,4.283333,5.736667,6.632308,7.823077,58399250.0,689.54,NaN,17.363231,0.616346,1.209923,0.749882
2025-03-31,-0.5,3.2,4.100000,4.210000,4.250000,4.453333,5.976667,6.827692,7.500000,55374920.0,693.76,NaN,18.521111,0.610678,1.519448,0.933009


In [11]:
import pandas as pd
from pandas_datareader import data as web

# ---------------------------------------------------------------------------
# 1. Series list  – only the five problematic entries were changed / replaced
# ---------------------------------------------------------------------------
fred_series = {
    # Growth & Income  -------------------------------------------------------
    "Real GDP growth (q/q SAAR)":          "A191RL1Q225SBEA",
    "Nominal GDP growth (q/q SAAR)":       "A191RP1Q027SBEA",
    "Real disposable income growth":       "A067RL1Q156SBEA",   # already SAAR %
    "Nominal disposable income growth":    "A067RP1Q027SBEA",   # already SAAR %

    # Labour-market & Prices  ------------------------------------------------
    "Unemployment rate":                   "UNRATE",            # monthly
    "CPI (price level)":                   "CPIAUCSL",          # monthly index

    # Interest Rates  --------------------------------------------------------
    "3-month Treasury rate":               "TB3MS",             # monthly
    "5-year Treasury yield":               "GS5",               # monthly
    "10-year Treasury yield":              "GS10",              # monthly
    "BBB corporate yield":                 "BAA",               # monthly
    "Mortgage rate (30yr fixed)":          "MORTGAGE30US",      # weekly
    "Prime rate":                          "DPRIME",            # daily

    # Asset Prices  ----------------------------------------------------------
    "Dow Jones Total Stock Market Index":  "BOGZ1FL073164013Q", # quarterly
    "House Price Index":                   "CSUSHPINSA",        # monthly, Jan-2000=100
    "Commercial Real Estate Price Index":  "BOGZ1FL075035503Q", # quarterly, $ millions
    "VIX (market volatility index)":       "VIXCLS"             # daily
}

# --------------------------------------
# 2. Pull data
# --------------------------------------
start_date = "1990-01-01"
raw = {}

for label, series_id in fred_series.items():
    try:
        raw[label] = web.DataReader(series_id, "fred", start_date)
    except Exception as e:
        print(f"Error loading {label}: {e}")

# --------------------------------------
# 3. Combine & resample to quarter-end
# --------------------------------------
all_df = pd.concat(raw.values(), axis=1)
all_df.columns = raw.keys()

q_idx = pd.date_range("1990-03-31", all_df.index.max(), freq="QE-DEC")
quarterly = pd.DataFrame(index=q_idx)

for col in all_df:
    s = all_df[col]
    if s.index.inferred_freq == "Q":
        quarterly[col] = s                      # already quarterly
    else:
        quarterly[col] = s.resample("QE-DEC").mean()

# --------------------------------------
# 4. Fix CPI inflation (q/q SAAR)
# --------------------------------------
cpi = quarterly["CPI (price level)"]
quarterly["CPI inflation rate"] = ((cpi / cpi.shift(1))**4 - 1) * 100

# --------------------------------------
# 5. Scale commercial-RE price index
# --------------------------------------
quarterly["Commercial Real Estate Price Index"] = (
    quarterly["Commercial Real Estate Price Index"] / 1_000
)

# --------------------------------------
# 6. House-keeping – drop the CPI level column
# --------------------------------------
quarterly.drop(columns=["CPI (price level)"], inplace=True)

# --------------------------------------
# 7. Sort / final NA cleanup
# --------------------------------------
quarterly = quarterly.sort_index().dropna(how="all")

# Preview
quarterly.tail(10)

# Optional: save
# quarterly.to_csv("fred_quarterly_data_fixed.csv")

,Real GDP growth (q/q SAAR),Nominal GDP growth (q/q SAAR),Real disposable income growth,Nominal disposable income growth,Unemployment rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,BBB corporate yield,Mortgage rate (30yr fixed),Prime rate,Dow Jones Total Stock Market Index,House Price Index,Commercial Real Estate Price Index,VIX (market volatility index),CPI inflation rate
2022-12-31,3.4,7.2,3.8,7.9,3.566667,4.040000,4.000000,3.830000,5.973333,6.664615,6.819672,38520600.0,296.545000,338.489,25.002656,4.110044
2023-03-31,2.8,6.6,10.9,15.3,3.533333,4.626667,3.800000,3.646667,5.600000,6.372308,7.693548,41136550.0,294.594000,338.612,20.684062,3.658320
2023-06-30,2.4,4.3,3.4,6.4,3.533333,5.073333,3.693333,3.593333,5.683333,6.510769,8.158730,44411470.0,305.162333,347.601,16.441250,2.998244
2023-09-30,4.4,7.7,1.4,4.1,3.666667,5.290000,4.313333,4.150000,5.973333,7.040000,8.432540,42788690.0,311.541667,339.944,15.013231,3.496197
2023-12-31,3.2,4.8,3.2,4.9,3.800000,5.283333,4.420000,4.440000,6.186667,7.303846,8.500000,47787470.0,312.017667,315.823,15.286250,2.794446
2024-03-31,1.6,4.7,5.6,9.2,3.833333,5.233333,4.123333,4.160000,5.733333,6.748462,8.500000,52402860.0,313.525667,306.404,13.712540,3.705673
2024-06-30,3.0,5.6,1.0,3.6,4.000000,5.243333,4.460000,4.443333,5.923333,6.995385,8.500000,53915680.0,323.343333,307.387,13.980308,2.756676
2024-09-30,3.1,5.0,0.2,1.8,4.166667,4.990000,3.790000,3.946667,5.620000,6.508462,8.437500,57046410.0,325.176667,302.611,17.066667,1.395962
2024-12-31,2.4,4.8,2.5,4.9,4.133333,4.400000,4.130000,4.283333,5.736667,6.632308,7.823077,58399250.0,323.729333,304.440,17.363231,3.033437
2025-03-31,-0.5,3.2,2.5,6.2,4.100000,4.210000,4.250000,4.453333,5.976667,6.827692,7.500000,55374920.0,325.448000,301.073,18.521111,3.784591
